# Draw! - Live Demo (with TensorRT)

The task of the robot is to drive around without collision to draw on a canvas.

The trained network should estimate in which direction it is save to drive:

- forward : both tracks have same speed value
- right : left track has set speed, right track is set 0
- left : right track has set speed, left track is set 0
- turn_right : left track has set speed, right has set -speed
- turn_left : right track has set speed, left has set -speed

In [ ]:
import torch
device = torch.device('cuda')

import torchvision
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_trt.pth'))

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, speed_slider]))

from jetbot import Robot

robot = Robot()

import torch.nn.functional as F
import time

def update(change):
    global speed_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model_trt(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    # Get the predicted class
    pred_class = torch.argmax(y, dim=1).item()

    speed = speed_slider.value
    
    if pred_class == 0:  # forward
        robot.forward(speed)
    elif pred_class == 1:  # left
        robot.left(speed)
    elif pred_class == 2:  # right
        robot.right(speed)
    elif pred_class == 3:  # turn_left
        robot.set_motors(-speed, speed)
    elif pred_class == 4:  # turn_right
        robot.set_motors(speed, -speed)
    else:
        robot.stop()
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

camera.observe(update, names='value')
